In [2]:
search_for = 127
start_from = 0
threads = 10
thread = 3
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004190683364868164
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 262209281
type: [1, 1, 1, 1, 127] 127
cases of this type: 260144641
100000 12.498179907534926
200000 21.18829627580991
300000 20.026742624360914
400000 20.108558021580762
500000 20.047973618994277
600000 20.714531845822584
700000 20.1392756311572
800000 19.791654668301703
900000 20.083561436185494
1000000 17.319821544992315
1100000 9.349251653961904
1200000 19.580737352853042
1300000 19.219624636733357
1400000 19.379114228139496
1500000 18.81963228290973
1600000 20.44602307364564
1700000 19.208363719151425
1800000 19.518878616560674
1900000 19.60897654419334
2000000 19.900080962249497
2100000 5.274263832870038
2200000 21.767443517840036
2300000 23.918143456682266
2400000 22.980159300662567
2500000 23.3428795558909
2600000 23.62941942907297
2700000 22.7989527604861
2800000 21.702933937326033
2900000 22.235201637667316
3000000 21.8

29400000 17.271160926475076
29500000 17.564659384832808
29600000 16.720460444634046
29700000 11.793845672252923
29800000 11.608899531045873
29900000 16.781650514350158
30000000 17.22819833355424
30100000 17.08497400090791
30200000 18.749006922259706
30300000 18.158995223810702
30400000 18.689185806554914
30500000 12.144930724458744
30600000 14.803029644646996
30700000 18.50880613639833
30800000 18.321182022234876
30900000 18.164946316385734
31000000 9.71648072974205
31100000 16.13904979107154
31200000 18.20215489874372
31300000 17.990941595339315
31400000 18.07743273027766
31500000 16.842253291192893
31600000 16.295856211864102
31700000 16.62394286904016
31800000 5.5651858688836455
31900000 16.95072071181488
32000000 16.4253453657705
32100000 16.595082281292633
32200000 16.961189998015772
32300000 17.934905418660023
32400000 17.800769903548954
32500000 16.137651562601807
32600000 10.598983512627857
32700000 17.86284871125082
32800000 18.075795002163144
32900000 18.46432653608144
330000

59100000 15.643203507898248
59200000 16.28012891685122
59300000 16.103750316615145
59400000 16.325552517503066
59500000 16.393293381533
59600000 16.052009533868826
59700000 15.998546125209504
59800000 15.788793996055066
59900000 8.75441320278505
60000000 12.989049322271969
60100000 14.062178032282715
60200000 14.38689708815222
60300000 14.108277943191847
60400000 13.05788287865112
60500000 6.823067732203536
60600000 14.17980906506078
60700000 14.051205003613482
60800000 13.728174530659567
60900000 13.848030779290545
61000000 8.44644560413
61100000 13.98635887534237
61200000 15.670023210173275
61300000 15.39198978599052
61400000 16.175776814350222
61500000 16.56395165669285
61600000 16.20926939665713
61700000 15.593366170974994
61800000 15.657009719564613
61900000 15.76887313728936
62000000 6.615685343506801
62100000 14.08945141110512
62200000 13.844749162948919
62300000 13.799187853061238
62400000 13.996401906832322
62500000 6.954331947994046
62600000 13.16144187801916
62700000 14.1080

88800000 7.048961203786073
88900000 10.187426444541124
89000000 12.483209498754793
89100000 8.9521647202051
89200000 7.945647500975014
89300000 11.680028725394584
89400000 11.257696173018699
89500000 7.091499924404026
89600000 11.824758791096057
89700000 11.836462821192644
89800000 12.085702797685657
89900000 12.823323298656875
90000000 13.141087821534095
90100000 12.9176726609328
90200000 13.165309659972156
90300000 13.246295899507382
90400000 12.896527818150814
90500000 12.71016347890681
90600000 12.288974549179354
90700000 11.899950662784667
90800000 12.08781887824294
90900000 5.069541396122064
91000000 11.896010886285891
91100000 11.959116577170555
91200000 4.019596187415895
91300000 12.035175697528983
91400000 12.034012371951968
91500000 5.700393535463786
91600000 12.380480533749191
91700000 11.896057579816985
91800000 11.679092856914577
91900000 12.490591971327689
92000000 12.989980125715482
92100000 13.023865761225215
92200000 13.132014440215185
92300000 12.725356803538
92400000

117900000 4.218915674236752
118000000 9.074590723034468
118100000 9.654040883558109
118200000 9.892903176184074
118300000 9.817856672876852
118400000 9.7355442091398
118500000 10.03247154571757
118600000 10.100269458121636
118700000 10.239959208101029
118800000 10.750464500209443
118900000 11.217383593373794
119000000 9.758597200518777
119100000 9.818598580203657
119200000 9.516776645175106
119300000 9.517934466393376
119400000 9.706802022306128
119500000 9.699889678259153
119600000 9.100243538692217
119700000 9.90760885541186
119800000 3.265160111687105
119900000 1.7916081147604825
120000000 6.882070275048862
120100000 9.385945924248407
120200000 9.181372585189964
120300000 9.448812895548873
120400000 9.486822398605462
120500000 9.496624686708277
120600000 9.57847210659011
120700000 9.917255993844021
120800000 9.965179176868315
120900000 11.557143419330282
121000000 11.028047271242936
121100000 10.56628510335719
121200000 10.34256313426316
121300000 10.029193215332551
121400000 10.208

146800000 7.688412694281328
146900000 8.055645748478792
147000000 8.132356937088968
147100000 8.021177498626786
147200000 8.158623175744665
147300000 8.046426625661208
147400000 8.881650640759037
147500000 8.860629185854377
147600000 9.240341735451418
147700000 8.462741421131511
147800000 8.145332665291821
147900000 8.097809342163897
148000000 8.276349728731784
148100000 8.099600079232376
148200000 7.805428088302486
148300000 7.725141101185203
148400000 3.8094713876522204
148500000 6.433272421673434
148600000 5.441916975293245
148700000 3.427207864408783
148800000 7.7974254664827045
148900000 7.948830798782651
149000000 7.877025369261307
149100000 7.880493465284537
149200000 8.02601012869267
149300000 7.919510754721703
149400000 8.345335981672735
149500000 8.756920510722981
149600000 8.801684874195345
149700000 8.52368207338972
149800000 8.1428494453374
149900000 7.820659738692767
150000000 7.933549347354318
150100000 7.967426945040654
150200000 7.6153104728740315
150300000 7.598623654

176100000 6.3516198965800115
176200000 6.464028472607683
176300000 6.272746069775027
176400000 6.435480692175665
176500000 6.196356343130603
176600000 6.102526069156134
176700000 5.876415562183626
176800000 4.699771715976183
176900000 3.9171819378186465
177000000 5.868325941223435
177100000 5.909674678665718
177200000 3.200803322736593
177300000 4.8456643311259
177400000 5.737603384752902
177500000 5.857628280472354
177600000 2.737459145743033
177700000 6.021849334587123
177800000 5.768946301770705
177900000 6.289518358817109
178000000 6.23279338880052
178100000 6.431771507420343
178200000 6.4103526659820425
178300000 6.284101060219331
178400000 6.202354278951461
178500000 6.077556615297443
178600000 6.125369142462348
178700000 5.745642751795804
178800000 5.489496572285823
178900000 2.763433518327471
179000000 5.771963326922677
179100000 5.650226813093584
179200000 5.422120582640719
179300000 2.337065217398266
179400000 5.492893202867711
179500000 5.859121668428493
179600000 4.85851444

205300000 2.0551092679238647
205400000 4.190183275938795
205500000 4.088448595149424
205600000 3.924654505257162
205700000 3.8855321107771217
205800000 3.8234987326602217
205900000 1.434460437158708
206000000 4.068490392638089
206100000 3.8986662567736388
206200000 3.8666259525956312
206300000 3.9126379817524075
206400000 4.138410207738335
206500000 1.7362018313696737
206600000 4.299949998025668
206700000 4.352190367344633
206800000 4.299597621839925
206900000 4.299984659076991
207000000 4.341168674009803
207100000 4.262128627997806
207200000 4.244386252342101
207300000 2.9156557195896378
207400000 3.103432372893216
207500000 3.9598958947011056
207600000 3.849759500013967
207700000 3.671934141977992
207800000 3.6760298207270545
207900000 2.887401481451705
208000000 2.3162583707945856
208100000 3.6592133073235966
208200000 3.6930775989705387
208300000 3.8067539759428253
208400000 3.8541837378483654
208500000 3.632067758205376
208600000 2.1368862958837864
208700000 4.137151498215701
2088

234100000 2.216416279196928
234200000 2.192499604474197
234300000 2.1493245165447608
234400000 2.072617429479653
234500000 2.103544621644229
234600000 0.7262469598833653
234700000 2.0111278433528006
234800000 2.0654051486120957
234900000 2.1000806353997534
235000000 2.138843237872003
235100000 2.1109928730092724
235200000 2.179128130744803
235300000 2.1125402861991103
235400000 0.9032110574710607
235500000 2.1802705810527265
235600000 2.1584212947409456
235700000 2.121419416278702
235800000 0.9342604371320791
235900000 2.0803299567713105
236000000 2.0663562932740906
236100000 1.9646262454143082
236200000 1.9505098009716895
236300000 1.9753995927380026
236400000 1.903554120011231
236500000 1.8253568526437858
236600000 1.0721836115157366
236700000 1.5138441838348353
236800000 1.818769427591921
236900000 1.8962035644691089
237000000 2.0037428339032335
237100000 1.9353802792829877
237200000 1.947829612604374
237300000 1.9419941715451459
237400000 1.826747281846051
237500000 1.0190904441753

262100000 0.0006600306340634823
best so far: 0
type: [1, 1, 127, 1, 1] 127
cases of this type: 16129
262200000 6.050914291938146e-05
best so far: 0
type: [1, 127, 1, 1, 1] 127
cases of this type: 127
best so far: 0
type: [127, 1, 1, 1, 1] 127
cases of this type: 1
best so far: 0
all cases: 262209281
best: 0
